### A08: Bootstrapping

In [98]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.linear_model import Ridge

## Avertising

In [16]:
ad= pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Advertising.csv")
ad.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [26]:
np.random.seed(0)

In [36]:
x1 = df.TV.values.reshape([-1,1])
x2 = df.radio.values.reshape([-1,1])
x3 = df.newspaper.values.reshape([-1,1])
y1= df.sales

unos=np.ones([n1,1])
X=np.hstack([unos,x1,x2,x3])
ols=sm.OLS(y1,X)
resultados=ols.fit()
resultados.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 20 Nov 2025   Prob (F-statistic):           1.58e-96
Time:                        17:07:14   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
n_boot=1000
coef_sample=np.zeros((n_boot,X.shape[1]))
for i in range(n_boot):
    sample_indices=np.random.choice(range(len(y1)), size=len(y1), replace=True)
    X_sample=X[sample_indices]
    y_sample=y1.iloc[sample_indices]
    ols=sm.OLS(y_sample, X_sample)
    resultados=ols.fit()
    coef_sample[i,:]=resultados.params
coef_means=np.mean(coef_sample, axis=0)
print("Coeficientes medios: ", coef_means)
#desviación
coef_stds=np.std(coef_sample, axis=0)
print("Desviación estándar: ", coef_stds)

Coeficientes medios:  [ 2.94333876e+00  4.56601140e-02  1.88704777e-01 -5.63751960e-04]
Desviación estándar:  [0.33469599 0.00189759 0.01081776 0.00647504]


## Default

In [46]:
defa=pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Default.csv")
defa.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [72]:
stu=(defa["student"]=="Yes").astype(int)
x1=defa["balance"].values.reshape(-1,1)
x3=stu.values.reshape(-1,1)
x2=defa["income"].values.reshape(-1,1)
y=defa["default"]== "Yes"
# Matriz de predictores
X3=np.hstack([x1,x2,x3])

modelo=LogisticRegression()
modelo.fit(X3,y)

#Intercepto y coeficientes
beta0_3=modelo.intercept_
beta1_3=modelo.coef_

print("Intercept: ",beta0_3)
print("Coeficientes: ",beta1_3)


Intercept:  [-10.90178652]
Coeficientes:  [[ 5.73059385e-03  3.96180616e-06 -6.12575695e-01]]


In [116]:
defa['student_Yes'] = pd.get_dummies(defa['student'], drop_first=True)
X3 = defa[['balance', 'income', 'student_Yes']]
y= defa['default']=='Yes'
X_multi = LogisticRegression()
X_multi.fit(X3, y)


LogisticRegression()

In [118]:
n_boot = 1000
coef_samples = np.zeros((n_boot, X3.shape[1] + 1))  

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_sample = X3.iloc[sample_indices]
    y_sample = y.iloc[sample_indices]

    logit = sm.Logit(y_sample, sm.add_constant(X_sample))
    resultados = logit.fit(disp=0)  # disp=0 para que no imprima cada iteración
    coef_samples[i, :] = resultados.params

coef_means = np.mean(coef_samples, axis=0)
coef_stds = np.std(coef_samples, axis=0)

print("Coeficientes medios:", coef_means)
print("Desviación estándar:", coef_stds)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [120]:
# Convertir X3 a float
X3 = X3.astype(float)
y = (defa['default'] == 'Yes').astype(int)

n_boot = 1000
coef_samples = np.zeros((n_boot, X3.shape[1] + 1))  # +1 para intercepto

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_sample = X3.iloc[sample_indices]
    y_sample = y.iloc[sample_indices]

    # statsmodels requiere float explícito
    X_sample_const = sm.add_constant(X_sample).astype(float)

    logit = sm.Logit(y_sample, X_sample_const)
    resultados = logit.fit(disp=0)
    coef_samples[i, :] = resultados.params

coef_means = np.mean(coef_samples, axis=0)
coef_stds = np.std(coef_samples, axis=0)

print("Coeficientes medios:", coef_means)
print("Desviación estándar:", coef_stds)


Coeficientes medios: [-1.09051367e+01  5.75297435e-03  3.02439644e-06 -6.44260215e-01]
Desviación estándar: [4.77378904e-01 2.30157001e-04 8.18560736e-06 2.35915996e-01]


## Avertising con penalización de Ridge (L2)

In [100]:
alpha = 1.0
coef_ridge = np.zeros((n_boot, X.shape[1]))

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y1)), size=len(y1), replace=True)
    X_sample = X[sample_indices]
    y_sample = y1.iloc[sample_indices]

    ridge = Ridge(alpha=alpha)
    ridge.fit(X_sample, y_sample)

    # Guardas intercepto y coeficientes si existen
    coef_ridge[i, 0] = ridge.intercept_
    coef_ridge[i, 1:] = ridge.coef_
    
coef_means_ridge = np.mean(coef_ridge, axis=0)
print("Coeficientes medios Ridge:", coef_means_ridge)

coef_stds_ridge = np.std(coef_ridge, axis=0)
print("Desviación estándar Ridge:", coef_stds_ridge)


Coeficientes medios Ridge: [14.18509753]
Desviación estándar Ridge: [0.69840443]
